In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
dict = pickle.load(open('./nums_detector.pickle', 'rb'))

print(dict)

{'x': [[0.520546555519104, 0.6225223541259766, 0.5548042058944702, 0.5314059257507324, 0.5657439231872559, 0.39597320556640625, 0.5913633108139038, 0.3044030964374542, 0.6225523352622986, 0.252106249332428, 0.4313318133354187, 0.28482043743133545, 0.5422664880752563, 0.2009216696023941, 0.6140134334564209, 0.2155720591545105, 0.6516286730766296, 0.24579893052577972, 0.436008483171463, 0.27567774057388306, 0.5731955170631409, 0.18492491543293, 0.6438739895820618, 0.23406177759170532, 0.6771804094314575, 0.2864888906478882, 0.4663742184638977, 0.2768646776676178, 0.6018471121788025, 0.1917218714952469, 0.6651273965835571, 0.24677006900310516, 0.6954826712608337, 0.30589085817337036, 0.5198541879653931, 0.29121118783950806, 0.6220865249633789, 0.21556179225444794, 0.6676642298698425, 0.2451898455619812, 0.6958702802658081, 0.28955429792404175], [0.520546555519104, 0.6225223541259766, 0.5548042058944702, 0.5314059257507324, 0.5657439231872559, 0.39597320556640625, 0.5913633108139038, 0.304

In [3]:
x = np.asarray(dict['x'])
labels = np.asarray(dict['labels'])

In [4]:
from sklearn.preprocessing import LabelEncoder
label_b = LabelEncoder()
labels = label_b.fit_transform(labels)

print(x.shape)
values, counts = np.unique(labels, return_counts=True)

for i in values:
    print(values[i], ':', counts[i])

(997, 42)
0 : 99
1 : 100
2 : 98
3 : 100
4 : 100
5 : 100
6 : 100
7 : 100
8 : 100
9 : 100


In [5]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, labels, train_size= 0.8, random_state = 42)

In [6]:
print(x_train.shape)

(797, 42)


In [7]:
import tensorflow as tf

batch_size = 128
num_classes = 10
epochs = 30

model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(42, 1)),
    tf.keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size=2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])
model.summary()




Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 40, 32)            128       
                                                                 
 conv1d_1 (Conv1D)           (None, 38, 64)            6208      
                                                                 
 max_pooling1d (MaxPooling1  (None, 19, 64)            0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 19, 64)            0         
                                                                 
 flatten (Flatten)           (None, 1216)              0         
                                                                 
 dense (Dense)               (None, 128)               155776    
                                                     

In [13]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
trained_model = model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = epochs, batch_size=batch_size)

Epoch 1/30


7/7 [==============================] - 2s 72ms/step - loss: 2.2936 - accuracy: 0.1092 - val_loss: 2.2702 - val_accuracy: 0.0550
Epoch 2/30
7/7 [==============================] - 0s 13ms/step - loss: 2.2378 - accuracy: 0.2158 - val_loss: 2.1998 - val_accuracy: 0.2700
Epoch 3/30
7/7 [==============================] - 0s 12ms/step - loss: 2.1611 - accuracy: 0.2685 - val_loss: 2.0940 - val_accuracy: 0.4000
Epoch 4/30
7/7 [==============================] - 0s 12ms/step - loss: 2.0667 - accuracy: 0.3450 - val_loss: 1.9652 - val_accuracy: 0.5500
Epoch 5/30
7/7 [==============================] - 0s 19ms/step - loss: 1.9334 - accuracy: 0.4291 - val_loss: 1.7704 - val_accuracy: 0.7400
Epoch 6/30
7/7 [==============================] - 0s 11ms/step - loss: 1.7473 - accuracy: 0.4680 - val_loss: 1.5455 - val_accuracy: 0.7550
Epoch 7/30
7/7 [==============================] - 0s 12ms/step - loss: 1.5374 - accuracy: 0.5445 - val_loss: 1.3201 - val_accuracy: 0.7200
Epoch 8/30
7/7 [=========

In [10]:
Labels_dict = {
        0 : '0',
        1 : '1',
        2 : '2',
        3 : '3',
        4 : '4',
        5 : '5',
        6 : '6',
        7 : '7',
        8 : '8',
        9 : '9',
        }


In [11]:
#Test the Model

import mediapipe as mp
import cv2
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

hands = mp_hands.Hands(static_image_mode = True, min_detection_confidence = 0.3, max_num_hands = 1)

cap = cv2.VideoCapture(0)
while (cap.isOpened()):
    
    data_aux = []
    x_ = []
    y_ = []

    ret, frame = cap.read()

    H, W, _ = frame.shape

    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    results = hands.process(frame_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_drawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mp_hands.HAND_CONNECTIONS,  # hand connections
                mp_drawing_styles.get_default_hand_landmarks_style(),
                mp_drawing_styles.get_default_hand_connections_style())

        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                x_.append(x)
                y_.append(y)

            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x)
                data_aux.append(y)

        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10
        
        # print(data_aux)
        t = np.asarray(data_aux).reshape(1, 42)
        # print(t)
        # print(t.shape)
        
        prediction = np.argmax(model.predict(t), axis=1)
        # print(prediction)
        predicted_character = Labels_dict[prediction[0]]
        
        print(predicted_character)
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)
        cv2.putText(frame, predicted_character, (300, 100), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (255, 0, 0), 3,
                    cv2.LINE_AA)

    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q') :
        break

cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 149ms/step
0
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 22ms/step
1
1/1 [==============================] - 0s 23ms/step
1
1/1 [==============================] - 0s 22ms/step
1
1/1 [==============================] - 0s 24ms/step
1
1/1 [==============================] - 0s 50ms/step
1
1/1 [==============================] - 0s 23ms/step
1
1/1 [==============================] - 0s 23ms/step
1
1/1 [==============================] - 0s 22ms/step
1
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 21ms/step
2
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 24ms/step
6
1/1 [======================

In [12]:
#Save the Model
model.save('nums_detection.keras')